# Instructor's side script for config file

In [ ]:
steps = [True, True, True, True, True, True, True, True, True, True, True, True]

In [ ]:
import subprocess
import os
import pandas as pd

if(steps[0]):
    steps[0] = False
    data = pd.read_csv('data.csv')
    data.columns = ['course', 'instructor']
    course = data['course'].to_string(index=False)
    instructor = data['instructor'].to_string(index=False)



    # Change directory to ~/.jupyter/
    os.chdir(os.path.expanduser("~/.jupyter/"))

    # Run the nbgrader generate_config command
    subprocess.run(["nbgrader", "generate_config"])


    print('--------------------------------------------------------')
    print('****************** Success!!!! *************************')
    print('--------------------------------------------------------')

else:
    print("\nYou already executed this step!\n")
    

In [ ]:
if(steps[1]):
    steps[1] = False
    with open(f'/home/{instructor}/.jupyter/nbgrader_config.py', 'r') as f:
        lines = f.readlines()

    new_lines = [
        f'\n\nc.CourseDirectory.root = \'/home/{instructor}/{course}\'\n',
        f'c.CourseDirectory.course_id = \'{course}\'\n',
        '\n',
        'c.GenerateFeedback.preprocessors = [\n',
        '    \'nbgrader.preprocessors.GetGrades\',\n',
        '    \'nbconvert.preprocessors.CSSHTMLHeaderPreprocessor\',\n',
        '    # remove: hidden tests from feedback:\n',
        '    #\'nbgrader.preprocessors.ClearHiddenTests\',\n',
        '    # remove tracebacks of hidden tests from feedback\n',
        '    #\'nbgrader.preprocessors.Execute\',\n',
        ']\n',
        '\n'
    ]

    lines[3:3] = new_lines

    with open('nbgrader_config.py', 'w') as f:
        f.writelines(lines)



    print('--------------------------------------------------------')
    print('******Configuration file updated successfully!**********')
    print('--------------------------------------------------------')
    
else:
    print("\nYou already executed this step!\n")

In [ ]:
if(steps[2]):
    steps[2] = False
    os.makedirs(os.path.expanduser(f'~/{course}'), exist_ok=True)
    # Change directory to ~/course
    os.chdir(os.path.expanduser(f"~/{course}"))

    print('--------------------------------------------------------')
    print('************ Course directory created! *****************')
    print('--------------------------------------------------------')
    
else:
    print("\nYou already executed this step!\n")


# Assignment Section Related Tasks

### Adding assignmet to DB

In [ ]:
if(steps[3]):
    steps[3] = False
    
    from datetime import date,timedelta
    assignment = input('Please Input the Assignment Name: ')
    today = date.today() + timedelta(days=10)
    duedate = today.strftime("%Y-%m-%d")
else:
    print("\nYou already executed this step!\n")



In [ ]:
if(steps[4]):
    steps[4] = False
    !nbgrader db assignment add {assignment} --duedate={duedate}
    os.makedirs(os.path.expanduser(f'~/{course}/source/{assignment}/'), exist_ok=True)
    print('--------------------------------------------------------')
    print(f'************ {assignment} added to the DB. ***********')
    print('--------------------------------------------------------')



    #print(f"{assignment} added to the DB.")

    
else:
    print("\nYou already executed this step!\n")



<h3 style="color:red"> Paste assignment file to course -> source -> assignment </h3>

### Generate assignment 

In [ ]:
if(steps[5]):
    steps[5] = False
    !nbgrader generate_assignment {assignment}
    #print(f"{assignment} generated.")

    print('--------------------------------------------------------')
    print(f'************ {assignment} generated       . ***********')
    print('--------------------------------------------------------')
    
else:
    print("\nYou already executed this step!\n")




## Run this script to insert bulk sultions from LMS to Grader 

In [ ]:
import zipfile
import os

if(steps[7]):
    steps[7] = False
    
    # path to the zip file
    zip_path = f"/home/{instructor}/solutions.zip"

    # create a directory to store the extracted files
    extract_dir = f"/home/{instructor}/{course}/submitted"
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    # open the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # loop through all the files in the zip
        for filename in zip_ref.namelist():
            # check if the file is a notebook assignment
            if filename.endswith('.ipynb'):
                # get the student ID from the filename
                student_id = filename.split('_')[-1].split('.')[0]
                # get the assignment number from the filename
                assignment_num = filename.split('_')[2]
                # get the new filename
                new_filename = filename.split('_')[-2] + '.ipynb'
                # create the student directory if it doesn't exist
                student_dir = os.path.join(extract_dir, student_id)
                if not os.path.exists(student_dir):
                    os.makedirs(student_dir)
                # create the assignment directory if it doesn't exist
                assignment_dir = os.path.join(student_dir, "Assignment"+assignment_num)
                if not os.path.exists(assignment_dir):
                    os.makedirs(assignment_dir)
                # extract the file to the assignment directory with the original filename
                zip_ref.extract(filename, assignment_dir)
                # rename the file with the new filename
                os.rename(os.path.join(assignment_dir, filename), os.path.join(assignment_dir, "Assignment"+new_filename))
                # add due date text file to the assignment directory
                with open(os.path.join(assignment_dir, 'timestamp.txt'), 'w') as f:
                    f.write('2023-06-11 12:00:00 UTC')

    print('------------------------------------------------------------------------')
    print(f'*** Student Solutions of {assignment} are uploaded Successfully. ******')
    print('------------------------------------------------------------------------')
else:
    print("\nYou already executed this step!\n")




## To auto grade all assignmets of this course

In [ ]:
if(steps[8]):
    steps[8] = False
    
    !nbgrader autograde {assignment}

    print('----------------------------------------------')
    print('All the assignments are graded successfully!!!!')
    print('----------------------------------------------')
else:
    print("\nYou already executed this step!\n")



## Generate HTML feeback with comments 

In [ ]:
if(steps[9]):
    steps[9] = False
    !nbgrader generate_feedback {assignment}
    print('---------------------------------------------------------')
    print('All the assignments feedback are created successfully!!!!')
    print('---------------------------------------------------------')
    
else:
    print("\nYou already executed this step!\n")



## Exporting Graders Details of each assignmnet 

In [ ]:
if(steps[10]):
    steps[10] = False
    
    !nbgrader export

    print('-----------------------------------------------------------')
    print(f'Result of {assignment} has been generated successfully!!!!')
    print('-----------------------------------------------------------')
    
else:
    print("\nYou already executed this step!\n")



## Script to download feedback files

In [ ]:
if(steps[11]):
    import os
    import shutil
    import zipfile

    def copy_html_files(parent_dir, course):
        # Set the name of the grandchild folder containing the HTML file
        grandchild_folder_name = f"{assignment}"

        # Loop through each child folder in the parent folder
        for child_folder_name in os.listdir(parent_dir):
            child_folder_path = os.path.join(parent_dir, child_folder_name)
            if os.path.isdir(child_folder_path):
                # Check if the grandchild folder exists within the child folder
                grandchild_folder_path = os.path.join(child_folder_path, grandchild_folder_name)
                if os.path.exists(grandchild_folder_path):
                    # Find the HTML file within the grandchild folder
                    html_file_path = None
                    for file_name in os.listdir(grandchild_folder_path):
                        if file_name.endswith(".html"):
                            html_file_path = os.path.join(grandchild_folder_path, file_name)
                            break
                    if html_file_path is not None:
                        # Rename the HTML file and copy it to the parent folder
                        grandchild_folder_name2 = grandchild_folder_name[-1:]
                        new_file_name = "Fall2023_{}_{}_{}.html".format(course, grandchild_folder_name2, child_folder_name)
                        new_file_path = os.path.join(parent_dir, "feedback_marked", new_file_name)

                        # Create the 'xyz' directory if it doesn't exist
                        os.makedirs(os.path.dirname(new_file_path), exist_ok=True)

                        shutil.copy(html_file_path, new_file_path)

        # Create a zip file of the 'xyz' directory
        zip_file_path = os.path.join(parent_dir, "feedback_marked.zip")
        with zipfile.ZipFile(zip_file_path, "w") as zip_file:
            for file_name in os.listdir(os.path.join(parent_dir, "feedback_marked")):
                file_path = os.path.join(parent_dir, "feedback_marked", file_name)
                zip_file.write(file_path, file_name)

        # Delete the 'xyz' directory
        shutil.rmtree(os.path.join(parent_dir, "feedback_marked"))

    # Example usage:
    parent_dir = f"/home/{instructor}/{course}/feedback/"
    copy_html_files(parent_dir, course)


    print('------------------------------------------------------------')
    print(f'******** {assignment} marked feedback generated. **********')
    print('----------/course/feedback/feedback_marked.zip  ------------')
    
else:
    print("\nYou already executed this step!\n")
